In [3]:
# Import the necessary libraries
import nltk
import evaluate
import numpy as np
import pandas as pd
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from sklearn.model_selection import train_test_split

In [4]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Load DataFrame
df = pd.read_json("hf://datasets/toughdata/quora-question-answer-dataset/Quora-QuAD.jsonl", lines=True)

# Convert DataFrame to Dataset
dataset = Dataset.from_pandas(df)

# Create DatasetDict
dataset_dict = DatasetDict({
    'train': dataset
})

# Print the result
print(dataset_dict)
dataset = dataset_dict["train"].train_test_split(test_size=0.2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 56402
    })
})


In [5]:
# Load the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

# Load the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
prefix = "answer the question: "
def preprocess_function(examples):
    """Add prefix to the sentences, tokenize the text, and set the labels"""
    # The "inputs" are the tokenized answer:
    inputs = [prefix + doc for doc in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    # The "labels" are the tokenized outputs:
    labels = tokenizer(text_target=examples["answer"], max_length=512, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/45121 [00:00<?, ? examples/s]

Map:   0%|          | 0/11281 [00:00<?, ? examples/s]

In [8]:
tokenized_dataset.keys()

dict_keys(['train', 'test'])

In [9]:
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))

In [12]:
# Set up Rouge score for evaluation
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

In [13]:
# Verify that GPU is available and clear CUDA cache
import torch
print(torch.cuda.is_available())  # Should return True
torch.cuda.empty_cache()

True


In [14]:
# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    push_to_hub=False
)

# Set up trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,3.169180,0.104445,0.018313,0.084938,0.094675


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=125, training_loss=3.451495849609375, metrics={'train_runtime': 354.8917, 'train_samples_per_second': 2.818, 'train_steps_per_second': 0.352, 'total_flos': 59370602323968.0, 'train_loss': 3.451495849609375, 'epoch': 1.0})

In [15]:
from transformers import pipeline

# Load the trained model and tokenizer into a pipeline for question-answering
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
The model 'T5ForConditionalGeneration' is not supported for question-answering. Supported models are ['AlbertForQuestionAnswering', 'BartForQuestionAnswering', 'BertForQuestionAnswering', 'BigBirdForQuestionAnswering', 'BigBirdPegasusForQuestionAnswering', 'BloomForQuestionAnswering', 'CamembertForQuestionAnswering', 'CanineForQuestionAnswering', 'ConvBertForQuestionAnswering', 'Data2VecTextForQuestionAnswering', 'DebertaForQuestionAnswering', 'DebertaV2ForQuestionAnswering', 'DistilBertForQuestionAnswering', 'ElectraForQuestionAnswering', 'ErnieForQuestionAnswering', 'ErnieMForQuestionAnswering', 'FalconForQuestionAnswering', 'FlaubertForQuestionAnsweringSimple', 'FNetForQuestionAnswering', 'FunnelForQuestionAnswering', 'GPT2ForQuestionAnswering', 'GPTNeoForQuestionAnswering', 'GPTNeoXForQuestionAnswering', 'GPTJForQuestionAnswering', 'IBert

In [16]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [18]:
predictions = trainer.predict(small_eval_dataset)

In [23]:
predicted_answers = tokenizer.batch_decode(predictions.predictions,skip_special_tokens=True)

In [34]:
len(predicted_answers)

1000

In [25]:
type(predicted_answers)

list

In [30]:
true_answers = small_eval_dataset['answer']

In [33]:
type(true_answers)

list

In [29]:
small_eval_dataset

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [36]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

def compute_metrics(predicted_answers, true_answers):
    # BLEU scorer requires a smoothing function to handle cases with 0 counts
    smoothie = SmoothingFunction().method4

    # Prepare BLEU scoring function
    def calculate_bleu(predicted_answers, true_answers):
        bleu_scores = []
        for pred, ref in zip(predicted_answers, true_answers):
            score = sentence_bleu([ref.split()], pred.split(), smoothing_function=smoothie)
            bleu_scores.append(score)
        return sum(bleu_scores) / len(bleu_scores)

    # Prepare ROUGE scoring function
    def calculate_rouge(predictions, references):
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        scores = {"rouge1": [], "rouge2": [], "rougeL": []}
        for pred, ref in zip(predicted_answers, true_answers):
            score = scorer.score(ref, pred)
            scores["rouge1"].append(score["rouge1"].fmeasure)
            scores["rouge2"].append(score["rouge2"].fmeasure)
            scores["rougeL"].append(score["rougeL"].fmeasure)
        return {key: sum(value) / len(value) for key, value in scores.items()}

    # Calculate BLEU and ROUGE scores
    bleu = calculate_bleu(predicted_answers, true_answers)
    rouge = calculate_rouge(predicted_answers, true_answers)

    # Combine the scores in a dictionary
    metrics = {
        "bleu": bleu,
        "rouge1": rouge["rouge1"],
        "rouge2": rouge["rouge2"],
        "rougeL": rouge["rougeL"]
    }

    return metrics

# Example usage
# predicted_answers = ["This is a predicted answer.", "Another predicted answer."]
# true_answers = ["This is the correct answer.", "Another correct answer."]

metrics = compute_metrics(predicted_answers, true_answers)
print(f"BLEU Score: {metrics['bleu']}")
print(f"ROUGE-1 Score: {metrics['rouge1']}")
print(f"ROUGE-2 Score: {metrics['rouge2']}")
print(f"ROUGE-L Score: {metrics['rougeL']}")


BLEU Score: 0.003737802682959363
ROUGE-1 Score: 0.10310794127031973
ROUGE-2 Score: 0.018071281211324976
ROUGE-L Score: 0.08388251685434296
